In [1]:
import numpy as np
import cv2
import serial
import config as cf
import threading
import time
from Router import DecodeCommand
from ActiveFunction import *

cf.RUN = True
cf.pause = False
WIDTH = 400
HEIGHT = 400
cf.command = ("nocommand", -1)

cf.ser = serial.Serial('/dev/ttyUSB1',115200)
font = cv2.FONT_HERSHEY_SIMPLEX
cf.global_img = np.zeros((400, 400, 3), np.uint8)

# LEDs parameters
N = 9
M = 4
cf.LEDs = np.zeros((M , N))
cf.LEDs_threshold = np.ones((M , N));
cf.LEDs_threshold = np.load("LEDs_threshold.npy")
cf.LEDs_min = np.ones((M , N))
cf.LEDs_max = np.ones((M , N))
cf.LEDs_MODE = np.zeros_like(cf.LEDs)
cf.LEDs_MODE_show = np.zeros_like(cf.LEDs)
cf.LEDs_weighted = np.zeros_like(cf.LEDs_MODE)
cf.LEDs_map = [0, 1, 2, 3]

# Speed parameters
# MOVE DIRECTION
cf.UP = [1, 1, 1, 1]
cf.DOWN = [-1, -1, -1, -1]
cf.LEFT = [-1, 1, -1, 1]
cf.RIGHT = [1, -1, 1, -1]
cf.TURN_LEFT = [-1, 1, 1, -1]
cf.TURN_RIGHT = [1, -1, -1, 1]
cf.PAUSE = [0, 0, 0, 0]
cf.cab = np.array([1, 1, 1, 1])
cf.DIRECTIONs = [cf.UP, cf.LEFT, cf.DOWN, cf.RIGHT, cf.PAUSE]

cf.speed = 200
cf.ratio = 0
cf.rotate = False

def read_inputs():
    print("read_inputs is started")
    while cf.RUN:
        try:
            # LEDS
            cf.message = cf.ser.readline().decode()[:-2] 
            cf.message_arr = cf.message.split(" ")
            if cf.message_arr[0] == 'LEDs':
                LEDs_arr = np.array(cf.message_arr[1:].copy()).astype(np.int32)
                LEDs_arr = LEDs_arr.reshape((M, N))
                cf.LEDs = LEDs_arr.copy()
                LEDs_MODE = np.zeros_like(cf.LEDs)
                LEDs_MODE[cf.LEDs >= cf.LEDs_threshold] = 1
                LEDs_MODE[cf.LEDs < cf.LEDs_threshold] = 0
                LEDs_MODE[0] = np.flip(LEDs_MODE[0].copy())
                LEDs_MODE[3] = np.flip(LEDs_MODE[3].copy())
                cf.LEDs_MODE_show = LEDs_MODE.copy()
                LEDs_MODE[2] = np.flip(LEDs_MODE[2].copy())
                cf.LEDs_MODE = LEDs_MODE.copy()
                weight = np.array([4, 3, 2, 1, 1, 1, 2, 3, 4])
                cf.LEDs_weighted = np.multiply(cf.LEDs_MODE, weight)
            # CTHTs
            for i in range(len(CTHTs)):
                cf.CTHT_values[i] = GPIO.input(CTHTs[i])
        
        except:
            pass
    GPIO.cleanup()
    print('read_inputs is stopped')


read_inputs_threading = threading.Thread(target=read_inputs)
read_inputs_threading.start()
time.sleep(0.1)
DecodeCommand_threading = threading.Thread(target=DecodeCommand)
DecodeCommand_threading.start()

def ping_to_arduino():
    message = "PUSH /"
    t1 = time.time()
    cf.ser.write(message.encode())
    while cf.message_arr[0] != 'PULL':
        time.sleep(0.005)
    print("Ping result:", time.time() - t1)
    
def show_leds_status(img):
    ## LEDS
    dis = 20
    for i in range(N):
        begin = int((WIDTH - ((N-1)*dis))/2)
        cv2.circle(img, ( begin+ dis*i, dis), 7, (255, 255, 255), -1)
        if cf.LEDs_MODE_show[0, i]:
            cv2.circle(img, ( begin+ dis*i, dis), 5, (0, 255, 0), -1)
    for i in range(N):
        begin = int((HEIGHT - ((N-1)*dis))/2)
        cv2.circle(img, (dis,  begin+ dis*i), 7, (255, 255, 255), -1)
        if cf.LEDs_MODE_show[1, i]:
            cv2.circle(img, (dis,  begin+ dis*i), 5, (0, 255, 0), -1)
    for i in range(N):
        begin = int((HEIGHT - ((N-1)*dis))/2)
        cv2.circle(img, (WIDTH - dis,  begin+ dis*i), 7, (255, 255, 255), -1)
        if cf.LEDs_MODE_show[3, i]:
            cv2.circle(img, (WIDTH - dis,  begin+ dis*i), 5, (0, 255, 0), -1)
    
    for i in range(N):
        begin = int((WIDTH - ((N-1)*dis))/2)
        cv2.circle(img, ( begin+ dis*i, HEIGHT - dis), 7, (255, 255, 255), -1)
        if cf.LEDs_MODE_show[2, i]:
            cv2.circle(img, ( begin+ dis*i, HEIGHT - dis), 5, (0, 255, 0), -1)
            
while cf.RUN:
    img = np.zeros((HEIGHT, WIDTH, 3), np.uint8)
    show_leds_status(img)
    cv2.imshow("robocon", img)
    k = cv2.waitKey(1)
    if k == ord('q'):
        quit()
    if k== 56:
        move(cf.UP, cf.speed)
    if k== 50:
        move(cf.DOWN, cf.speed)
    if k== 52:
        move(cf.LEFT, cf.speed)
    if k== 54:
        move(cf.RIGHT, cf.speed)
    if k== ord('o'):
        move(cf.TURN_LEFT, cf.speed)
    if k== ord('p'):
        move(cf.TURN_RIGHT, cf.speed)
    if k== 32:
        pause()
    if k== ord('w'):
        set_command("go_with_line",[0])
    if k== ord('a'):
        set_command("go_with_line", [1])
    if k== ord('d'):
        set_command("go_with_line", [3])
    if k== ord('s'):
        set_command("go_with_line", [2])
    if k== ord('e'):
        set_command("adj_car", 0)
    if k== ord('r'):
        set_command("adj_car", 1)
    if k== ord('t'):
        set_command("run_router", -1)
    if k == ord('z'):
        cf.LEDs_min = cf.LEDs.copy()
        print("min:") 
        print(cf.LEDs_min)
    if k == ord('x'):
        cf.LEDs_max[0] = cf.LEDs[0].copy()
        print("max UP:")
        print( cf.LEDs_max[0])
    if k == ord('c'):
        cf.LEDs_max[1] = cf.LEDs[1].copy()
        print("max LEFT:")
        print( cf.LEDs_max[1])
    if k == ord('v'):
        cf.LEDs_max[2] = cf.LEDs[2].copy()
        print("max RIGHT:")
        print( cf.LEDs_max[2])
    if k == ord('b'):
        cf.LEDs_max[3] = cf.LEDs[3].copy()
        print("max BOTTOM:")
        print( cf.LEDs_max[3])
    if k == ord ('n'):
        cf.LEDs_threshold = (cf.LEDs_min +cf.LEDs_max)/2
        np.save("LEDs_threshold.npy", cf.LEDs_threshold)
        print("Hieu:")
        print( -cf.LEDs_min +cf.LEDs_max)
        print("LEDs_threshold:")
        print(cf.LEDs_threshold)
    
cv2.destroyAllWindows()

read_inputs is started
DecodeCommand threading started!
adjust_car 0
1 1
[-1  1] [ 1 -1] [-1, 1, 1, -1]
1 1
[-1  1] [ 1 -1] [-1, 1, 1, -1]
pause
adjust_car 0
1 -1
[-1  1] [-1  1] [-1, 1, -1, 1]
1 -1
[-1  1] [-1  1] [-1, 1, -1, 1]
1 -1
[-1  1] [-1  1] [-1, 1, -1, 1]
pause
adjust_car 0
-1 1
[ 1 -1] [ 1 -1] [1, -1, 1, -1]
-1 1
[ 1 -1] [ 1 -1] [1, -1, 1, -1]
-1 0
[0 0] [-1  1] [0, 0, -1, 1]
pause
adjust_car 0
-1 1
[ 1 -1] [ 1 -1] [1, -1, 1, -1]
-1 0
[0 0] [-1  1] [0, 0, -1, 1]
-1 1
[ 1 -1] [ 1 -1] [1, -1, 1, -1]
-1 1
[ 1 -1] [ 1 -1] [1, -1, 1, -1]
-1 1
[ 1 -1] [ 1 -1] [1, -1, 1, -1]
-1 1
[ 1 -1] [ 1 -1] [1, -1, 1, -1]
-1 1
[ 1 -1] [ 1 -1] [1, -1, 1, -1]
-1 0
[0 0] [-1  1] [0, 0, -1, 1]
-1 0
[0 0] [-1  1] [0, 0, -1, 1]
-1 -1
[ 1 -1] [-1  1] [1, -1, -1, 1]
1 -1
[-1  1] [-1  1] [-1, 1, -1, 1]
0 -1
[ 1 -1] [0 0] [1, -1, 0, 0]
-1 -1
[ 1 -1] [-1  1] [1, -1, -1, 1]
pause
adjust_car 0
-1 -1
[ 1 -1] [-1  1] [1, -1, -1, 1]
-1 -1
[ 1 -1] [-1  1] [1, -1, -1, 1]
-1 -1
[ 1 -1] [-1  1] [1, -1, -1, 1]
-1 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: RuntimeWarning: No channels have been set up yet - nothing to clean up!  Try cleaning up at the end of your program instead!


read_inputs is stopped
DecodeCommand threading stopped!
